<a href="https://colab.research.google.com/github/andremonroy/stanWeinstein/blob/main/RS_Acciones_SectoresFuertes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lee desde Google Sheets los ETFs líderes desde RS_Sectores.ipynb

In [ ]:
# CELDA 1 – Instalación, aplicar formato visual usando gspread-formatting
!pip install --quiet yfinance gspread oauth2client
!pip install --quiet gspread-formatting
!pip install --upgrade gspread-formatting


In [ ]:
# CELDA 2 – Librerías y autenticación
import pandas as pd, numpy as np, yfinance as yf, datetime, gspread
from google.colab import auth
auth.authenticate_user()

import google.auth
creds, _ = google.auth.default()
client = gspread.authorize(creds)

# CONFIGURACIÓN DEL GOOGLE SHEETS
spreadsheet_id = "19WW_XIkvM0VU1W_NYDBEZwU6NxYvkTAb2Qn_E7ONAyw"
sheet_name     = "Hoja 1"  # contiene RS de ETFs

# Cargar hoja con sectores
sheet_sectores = client.open_by_key(spreadsheet_id).worksheet(sheet_name)
df_sectores    = pd.DataFrame(sheet_sectores.get_all_records())

# Seleccionar 2 sectores más fuertes
top_etfs = df_sectores.sort_values("RS", ascending=False).head(3)['ETF'].tolist()
print("🏆 Sectores más fuertes:", top_etfs)


🏆 Sectores más fuertes: ['XLK', 'XLI', 'XLU']


In [ ]:
# CELDA 3 – Configurar universo de acciones (personalizable)
sector_map = {
    'XLK': ['AAPL', 'MSFT', 'NVDA', 'AVGO', 'ADBE', 'CRM', 'AMD'],
    'XLI': ['HON', 'GE', 'UPS', 'CAT', 'UNP', 'DE', 'LMT']
}
'''
acciones = []
for etf in top_etfs:
    acciones += sector_map.get(etf, [])
'''
# Creamos una lista que relacione cada acción con su ETF (sector)
acciones_info = []

for etf in top_etfs:
    for ticker in sector_map.get(etf, []):
        acciones_info.append({'ETF': etf, 'Ticker': ticker})

# Convertimos esa lista en un DataFrame
df_acciones = pd.DataFrame(acciones_info)

# Extraemos solo la lista de tickers
acciones = df_acciones['Ticker'].tolist()

acciones = list(set(acciones))  # Eliminar duplicados
print(f"🔍 Acciones a analizar ({len(acciones)}):", acciones)

🔍 Acciones a analizar (14): ['CAT', 'MSFT', 'CRM', 'DE', 'GE', 'AVGO', 'AMD', 'ADBE', 'NVDA', 'AAPL', 'UPS', 'LMT', 'HON', 'UNP']


In [ ]:
# CELDA 4 – Cálculo de RS compuesto
today     = datetime.date.today()
start     = today - datetime.timedelta(weeks=60)
benchmark = yf.download('SPY', start=start, end=today, interval='1wk')['Close']
data      = yf.download(acciones, start=start, end=today, interval='1wk')['Close']

# Asegurar que 'data' tenga formato de DataFrame aunque sea 1 ticker
if isinstance(data, pd.Series):
    data = data.to_frame()

# ⚠️ Filtrar columnas con más de 80% NaNs
threshold = int(0.8 * data.shape[0])
data = data.dropna(axis=1, thresh=threshold)

# Actualizar lista de acciones válidas
acciones = list(data.columns)
print("✅ Acciones con datos válidos para análisis:", acciones)

# Ventanas y pesos
windows = {'3m':13, '6m':26, '9m':39, '12m':52}
weights = {'3m':0.40, '6m':0.30, '9m':0.20, '12m':0.10}
'''
scores = pd.Series(0.0, index=acciones)

for lbl, wks in windows.items():
    rel     = data.pct_change(wks)
    rel_rs  = rel.div(benchmark.pct_change(wks), axis=0)
    latest  = rel_rs.iloc[-1]
    scores += latest * weights[lbl]
    '''
scores = pd.Series(0.0, index=acciones)

data = data.loc[benchmark.index]
for lbl, wks in windows.items():
    try:
        print(f"\n📈 Procesando ventana: {lbl} ({wks} semanas)")
        rel = data.pct_change(wks)
        spy_returns = benchmark.pct_change(wks).squeeze()  # Asegura Serie

        # Expandimos SPY a todas las columnas
        spy_df = pd.DataFrame({ticker: spy_returns for ticker in rel.columns})

        rel_rs = rel / spy_df
        latest = rel_rs.iloc[-1]
        print("✔️ Últimos valores de rel_rs:")
        print(latest)

        latest = latest.dropna()
        print("🎯 Tickers con datos válidos en esta ventana:", latest.index.tolist())

        scores[latest.index] += latest * weights[lbl]
    except Exception as e:
        print(f"⚠️ Error calculando RS para ventana {lbl}: {e}")



rs_scaled = 100 * (scores - scores.min()) / (scores.max() - scores.min())
rs_scaled = rs_scaled.round(1)

print("\n✅ RS final calculado para:", rs_scaled.dropna().index.tolist())
print("📊 Valores RS:", rs_scaled.dropna().to_dict())


print("📅 Benchmark SPY tiene:", len(benchmark), "registros")
print(benchmark.tail())

print("📅 Data acciones shape:", data.shape)
print("Fechas:", data.index.min(), "a", data.index.max())



print("🎯 RS calculado para:", rs_scaled.dropna().index.tolist())
print("🔢 RS calculado para:", rs_scaled.dropna().to_dict())



/tmp/ipython-input-1089808862.py:4: FutureWarning: YF.download() has changed argument auto_adjust default to True
  benchmark = yf.download('SPY', start=start, end=today, interval='1wk')['Close']
[*********************100%***********************]  1 of 1 completed
/tmp/ipython-input-1089808862.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data      = yf.download(acciones, start=start, end=today, interval='1wk')['Close']
[*********************100%***********************]  14 of 14 completed


✅ Acciones con datos válidos para análisis: ['AAPL', 'ADBE', 'AMD', 'AVGO', 'CAT', 'CRM', 'DE', 'GE', 'HON', 'LMT', 'MSFT', 'NVDA', 'UNP', 'UPS']

📈 Procesando ventana: 3m (13 semanas)
✔️ Últimos valores de rel_rs:
Ticker
AAPL    0.210483
ADBE   -0.952292
AMD     5.906463
AVGO    3.559883
CAT     2.766827
CRM    -0.677078
DE      0.326473
GE      2.356950
HON     0.342751
LMT    -0.813688
MSFT    1.833014
NVDA    4.462288
UNP     0.269077
UPS    -0.811412
Name: 2025-08-04 00:00:00, dtype: float64
🎯 Tickers con datos válidos en esta ventana: ['AAPL', 'ADBE', 'AMD', 'AVGO', 'CAT', 'CRM', 'DE', 'GE', 'HON', 'LMT', 'MSFT', 'NVDA', 'UNP', 'UPS']

📈 Procesando ventana: 6m (26 semanas)
✔️ Últimos valores de rel_rs:
Ticker
AAPL    -1.837452
ADBE    -3.825128
AMD     11.314666
AVGO     5.821247
CAT      3.551546
CRM     -3.923563
DE       1.826358
GE       6.155318
HON      1.504418
LMT     -0.578517
MSFT     5.490098
NVDA     6.796129
UNP     -1.267087
UPS     -3.956586
Name: 2025-08-04 00:00:

In [ ]:
# CELDA 5 – Evaluar criterios de Weinstein
criterios = []

for ticker in acciones:
    precios = data[ticker].dropna()
    if len(precios) < 31:
        criterios.append([ticker, np.nan, np.nan, np.nan, 'NO DATA'])
        continue
    mm30      = precios.rolling(30).mean()
    actual    = precios.iloc[-1]
    media_30  = mm30.iloc[-1]
    pendiente = mm30.iloc[-1] - mm30.iloc[-2]
    cumple    = (actual > media_30) and (pendiente > 0)
    criterios.append([ticker, actual, round(media_30,2), round(pendiente,2), '✅' if cumple else '❌'])

df_criterios = pd.DataFrame(criterios, columns=['Ticker', 'Precio actual', 'MM30', 'Pendiente MM30', 'Cumple Weinstein'])

# Unir con RS
df_rs = pd.DataFrame({'Ticker': rs_scaled.index, 'RS': rs_scaled.values})

'''
acciones_df = df_rs.merge(df_criterios, on='Ticker')
acciones_df = acciones_df.sort_values('RS', ascending=False)
acciones_df.reset_index(drop=True, inplace=True)
'''

# Limpiamos los tickers en todos los DataFrames
df_rs['Ticker'] = df_rs['Ticker'].str.strip().str.upper()
df_criterios['Ticker'] = df_criterios['Ticker'].str.strip().str.upper()
df_acciones['Ticker'] = df_acciones['Ticker'].str.strip().str.upper()

# Versión con ETF y orden por fuerza + Weinstein
# Combinamos los datos: RS + criterios técnicos + ETF
acciones_df = df_rs.merge(df_criterios, on='Ticker').merge(df_acciones, on='Ticker')

# Ordenamos: primero los que cumplen Weinstein, luego por mayor RS
acciones_df = acciones_df.sort_values(['Cumple Weinstein', 'RS'], ascending=[False, False])
acciones_df.reset_index(drop=True, inplace=True)

acciones_df

,Ticker,RS,Precio actual,MM30,Pendiente MM30,Cumple Weinstein,ETF
0,UNP,23.8,222.059998,228.94,-0.02,❌,XLI
1,AAPL,20.4,203.350006,213.09,-1.10,❌,XLK
2,LMT,14.8,423.700012,456.95,-1.26,❌,XLI
3,CRM,6.3,252.320007,279.31,-2.15,❌,XLK
4,UPS,0.1,85.019997,103.77,-1.15,❌,XLI
5,ADBE,0.0,338.850006,395.18,-2.24,❌,XLK
6,AMD,100.0,176.779999,118.88,2.02,✅,XLK
7,AVGO,90.5,297.720001,228.00,2.49,✅,XLK
8,NVDA,82.0,180.000000,134.99,1.47,✅,XLK
9,GE,79.9,276.230011,220.59,3.50,✅,XLI


In [ ]:
# Revisamos que los índices sean iguales
print("Comparación de Tickers comunes:", set(rs_scaled.index) & set(df_criterios['Ticker']))


Comparación de Tickers comunes: {'CAT', 'MSFT', 'CRM', 'DE', 'GE', 'AVGO', 'AMD', 'ADBE', 'NVDA', 'AAPL', 'UPS', 'LMT', 'HON', 'UNP'}


In [ ]:
# CELDA 6 – Exportar resultados a Google Sheets (RS Acciones)
try:
    client.open_by_key(spreadsheet_id).del_worksheet(client.open_by_key(spreadsheet_id).worksheet("RS Acciones"))
except:
    pass

new_sheet = client.open_by_key(spreadsheet_id).add_worksheet(title="RS Acciones", rows="100", cols="10")

# 🔧 Reemplazar NaN y valores no válidos
acciones_df = acciones_df.replace([np.inf, -np.inf, np.nan], 'N/A')

# Exportar
new_sheet.update([acciones_df.columns.values.tolist()] + acciones_df.values.tolist())

print("✅ Datos exportados a hoja 'RS Acciones' en Google Sheets.")


✅ Datos exportados a hoja 'RS Acciones' en Google Sheets.


In [ ]:
# CELDA 7 – Aplicar formato visual a la hoja "RS Acciones"

'''
from gspread_formatting import *
from gspread_formatting.models import DataValidation

# Reopen the worksheet to ensure we have the correct reference
ws = client.open_by_key(spreadsheet_id).worksheet("RS Acciones")

# Activa filtros visuales
set_frozen(ws, rows=1)
set_data_validation_for_cell_range(ws, 'A1:G1', DataValidation(condition_type='ONE_OF_LIST', values=[]))  # activa filtros visuales

# Negrita para encabezado
fmt_encabezado = cellFormat(
    backgroundColor=color(0.9, 0.9, 0.9),
    textFormat=textFormat(bold=True, fontSize=11),
    horizontalAlignment='CENTER'
)
format_cell_range(ws, 'A1:H1', fmt_encabezado)

# Escala de colores para columna RS (columna B = columna 2)
format_cell_range(ws, 'B2:B100', ColorScaleRule(
    minpoint=ColorStyle(color='FFAAAA'),
    midpoint=ColorStyle(color='FFFFAA'),
    maxpoint=ColorStyle(color='AAFFAA')
))

# Colores para “Cumple Weinstein”
rule_si = BooleanRule(condition='TEXT_EQ', values=['✅'],
    format=cellFormat(backgroundColor=color(0.8, 1.0, 0.8)))
rule_no = BooleanRule(condition='TEXT_EQ', values=['❌'],
    format=cellFormat(backgroundColor=color(1.0, 0.8, 0.8)))

format_cell_ranges(ws, 'G2:G100', ConditionalFormatRule(rules=[rule_si, rule_no]))

print("🎨 Formato visual aplicado exitosamente.")


'''

'\nfrom gspread_formatting import *\nfrom gspread_formatting.models import DataValidation\n\n# Reopen the worksheet to ensure we have the correct reference\nws = client.open_by_key(spreadsheet_id).worksheet("RS Acciones")\n\n# Activa filtros visuales\nset_frozen(ws, rows=1)\nset_data_validation_for_cell_range(ws, \'A1:G1\', DataValidation(condition_type=\'ONE_OF_LIST\', values=[]))  # activa filtros visuales\n\n# Negrita para encabezado\nfmt_encabezado = cellFormat(\n    backgroundColor=color(0.9, 0.9, 0.9),\n    textFormat=textFormat(bold=True, fontSize=11),\n    horizontalAlignment=\'CENTER\'\n)\nformat_cell_range(ws, \'A1:H1\', fmt_encabezado)\n\n# Escala de colores para columna RS (columna B = columna 2)\nformat_cell_range(ws, \'B2:B100\', ColorScaleRule(\n    minpoint=ColorStyle(color=\'FFAAAA\'),\n    midpoint=ColorStyle(color=\'FFFFAA\'),\n    maxpoint=ColorStyle(color=\'AAFFAA\')\n))\n\n# Colores para “Cumple Weinstein”\nrule_si = BooleanRule(condition=\'TEXT_EQ\', values=[\'✅\